In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lingfengzhang/pediatric-polymicrogyria-mri-dataset")

print("Path to dataset files:", path)

100%|██████████| 709M/709M [00:11<00:00, 65.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2


In [ ]:
pip install torch torchvision matplotlib pillow scikit-learn


In [ ]:
import os
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import models
import random
from sklearn.model_selection import train_test_split


In [ ]:
from torchvision import transforms

# Define preprocessing transformations for test data (only resizing and normalization)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Define transformations for training data (include augmentation)
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomVerticalFlip(),    # Randomly flip images vertically
    transforms.RandomRotation(30),      # Rotate images by up to 30 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Adjust brightness/contrast
    transforms.Resize((224, 224)),      # Resize to 224x224 after augmentations
    transforms.ToTensor(),              # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])



In [ ]:
from torch.utils.data import Dataset
from PIL import Image

class PolymicrogyriaDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        # Open image
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
# Create Dataset objects for train and test sets
train_dataset = PolymicrogyriaDataset(train_paths, train_labels, transform=train_transform)
test_dataset = PolymicrogyriaDataset(test_paths, test_labels, transform=test_transform)


NameError: name 'train_paths' is not defined

In [ ]:
from torch.utils.data import DataLoader

# Create DataLoader objects for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


NameError: name 'train_dataset' is not defined

In [ ]:
# Verify dataset and dataloader
print(f"Number of batches in train loader: {len(train_loader)}")
print(f"Number of batches in test loader: {len(test_loader)}")

# Visualize a batch of images to ensure transformations are applied
import matplotlib.pyplot as plt

# Grab a batch of data
data_iter = iter(train_loader)
images, labels = next(data_iter)

# Convert tensor images back to numpy format for display
images = images.numpy().transpose((0, 2, 3, 1))  # Convert from (N, C, H, W) to (N, H, W, C)

# Plot a few images
fig, axes = plt.subplots(1, 5, figsize=(12, 12))
for i in range(5):
    axes[i].imshow(images[i])
    axes[i].axis('off')
plt.show()


NameError: name 'train_loader' is not defined

In [ ]:
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define base dataset path
base_path = '/root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2'
ppmr_path = os.path.join(base_path, 'PPMR')

# Collect image paths and labels
image_paths = []
labels = []

# Define a mapping for directories
main_dir_labels = {
    "PMGControlsEditedDec2021": "control",  # Healthy cases
    "PMGstudycaseslabelled": "polymicrogyria"  # PMG cases
}

if os.path.exists(ppmr_path) and os.path.isdir(ppmr_path):
    print(f"Processing PPMR directory: {ppmr_path}")
    for main_dir, label_name in main_dir_labels.items():
        main_dir_path = os.path.join(ppmr_path, main_dir)

        if os.path.isdir(main_dir_path):
            print(f"Processing main directory: {main_dir_path}")
            for patient_dir in os.listdir(main_dir_path):
                patient_path = os.path.join(main_dir_path, patient_dir)

                if os.path.isdir(patient_path):
                    for sub_dir in os.listdir(patient_path):
                        sub_dir_path = os.path.join(patient_path, sub_dir)

                        if os.path.isdir(sub_dir_path):
                            for fname in os.listdir(sub_dir_path):
                                if fname.endswith(('.jpg', '.png', '.jpeg')):
                                    file_path = os.path.join(sub_dir_path, fname)
                                    image_paths.append(file_path)
                                    labels.append(label_name)
        else:
            print(f"Main directory not found: {main_dir_path}")
else:
    print("PPMR directory not found.")

# Check if any files were found
print(f"Total images found: {len(image_paths)}")

# Label encoding
if image_paths:
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)

    # Split dataset into train and test
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        image_paths, encoded_labels, test_size=0.2, random_state=42
    )
    print(f"Train set: {len(train_paths)}, Test set: {len(test_paths)}")
else:
    print("No images found. Ensure the directory structure and file extensions are correct.")


Processing PPMR directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR
Processing main directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR/PMGControlsEditedDec2021
Processing main directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR/PMGstudycaseslabelled
Total images found: 15056
Train set: 12044, Test set: 3012


In [ ]:
class cDCMLoss(nn.Module):
    def __init__(self, margin=5.0, alpha=1.0):
        super(cDCMLoss, self).__init__()
        self.margin = margin
        self.alpha = alpha

    def forward(self, outputs, labels, center):
        distances = torch.norm(outputs - center, dim=1)
        normal_loss = (1 - labels) * distances
        anomaly_loss = (
            labels * (torch.clamp(self.margin - distances, min=0) +
                      1 / (1 + torch.exp(distances - self.margin)))
        )
        loss = torch.mean(normal_loss + self.alpha * anomaly_loss)
        return loss


NameError: name 'nn' is not defined

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

def train_cdcmloss(model, data, labels, n_folds=5, margin=5.0, alpha=1.0):
    kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    results = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(data)):
        print(f"Fold {fold + 1}/{n_folds}")

        train_data = data[train_idx]
        train_labels = labels[train_idx]
        val_data = data[val_idx]
        val_labels = labels[val_idx]

        # Define model, optimizer, and scheduler
        model = model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
        criterion = cDCMLoss(margin=margin, alpha=alpha)
        center = torch.zeros((1, model.fc.out_features)).to(device)  # Adjust for model output

        # Training Loop
        for epoch in range(epochs):  # Apply early stopping as needed
            model.train()
            # Training logic here

        # Validation
        model.eval()
        # Evaluate AUCROC, recall, F2, etc.
        results.append((fold_aucroc, fold_recall, fold_f2))

    return results


In [ ]:
from sklearn.metrics import recall_score, fbeta_score, roc_auc_score

def evaluate_model(model, data_loader, labels):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs.to(device))
            preds = torch.argmax(outputs, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    recall = recall_score(true_labels, predictions, average="binary")
    f2 = fbeta_score(true_labels, predictions, beta=2, average="binary")
    aucroc = roc_auc_score(true_labels, predictions)
    return recall, f2, aucroc


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, MobileNetV2, EfficientNetB0

def dilated_conv_block(x, filters, kernel_size, dilation_rate):
    x = layers.Conv2D(filters, kernel_size, dilation_rate=dilation_rate, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def squeeze_and_excitation_block(x, reduction_ratio=16):
    channels = x.shape[-1]
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Dense(channels // reduction_ratio, activation='relu')(se)
    se = layers.Dense(channels, activation='sigmoid')(se)
    se = layers.Reshape((1, 1, channels))(se)
    x = layers.Multiply()([x, se])
    return x

def custom_cnn(input_shape=(224, 224, 3)):
    inputs = layers.Input(shape=input_shape)

    # First block: Dilated Convolutions
    x = dilated_conv_block(inputs, 32, (3, 3), dilation_rate=1)
    x = dilated_conv_block(x, 64, (3, 3), dilation_rate=2)

    # Squeeze-and-Excitation Block
    x = squeeze_and_excitation_block(x)

    # More layers or blocks as required
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(10, activation='softmax')(x)

    model = models.Model(inputs, x)
    return model


In [ ]:
def integrate_model(base_model, input_shape=(224, 224, 3)):
    # Freeze the base model to use it as a feature extractor
    base_model.trainable = False

    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs, training=False)

    # Add custom CNN layers after the base model
    x = dilated_conv_block(x, 128, (3, 3), dilation_rate=1)
    x = squeeze_and_excitation_block(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(10, activation='softmax')(x)

    model = models.Model(inputs, x)
    return model

# ResNet50
resnet50_model = integrate_model(ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3)))

# MobileNetV2
mobilenetv2_model = integrate_model(MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3)))

# EfficientNetB0
efficientnetb0_model = integrate_model(EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3)))


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define EarlyStopping callback to stop training if validation loss doesn't decrease
# after 4,500 iterations. Set patience in terms of epochs.

# Assuming you have a fixed batch size and you're interested in the number of iterations
batch_size = 32
max_iterations = 4500

# Set patience based on the number of iterations per epoch
# For example, if your dataset has 1000 samples, you would get 1000 // 32 = ~31 iterations per epoch.
# To hit 4500 iterations, we calculate the number of epochs required:
iterations_per_epoch = len(X_train) // batch_size  # This should be the number of iterations per epoch
epochs_until_stop = max_iterations // iterations_per_epoch  # This gives you the number of epochs for 4500 iterations

# EarlyStopping with patience based on epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=epochs_until_stop, restore_best_weights=True)

# Now let's modify the training loop to use early stopping
kfold = StratifiedKFold(n_splits=5, shuffle=True)

for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
    print(f"Training fold {fold + 1}")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Create data generators
    train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
    val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

    # Choose a model (ResNet50, MobileNetV2, or EfficientNetB0)
    model = resnet50_model  # Example with ResNet50, you can switch between models

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model with EarlyStopping
    model.fit(train_generator, validation_data=val_generator, epochs=400, callbacks=[early_stopping])

    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(val_generator)
    print(f"Fold {fold + 1} - Validation Accuracy: {val_accuracy * 100:.2f}%")



NameError: name 'X_train' is not defined

In [ ]:
!git clone https://github.com/nkicsl/NKUT


Cloning into 'NKUT'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 164 (delta 0), reused 0 (delta 0), pack-reused 161 (from 1)
Receiving objects: 100% (164/164), 1.05 MiB | 3.03 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [ ]:
%cd NKUT


/content/NKUT


In [ ]:
!pip install -r requirements.txt


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!pip install torch torchvision


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import Recall

# Define base dataset path
base_path = '/root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2'
ppmr_path = os.path.join(base_path, 'PPMR')

# Collect image paths and labels
image_paths = []
labels = []

# Define a mapping for directories
main_dir_labels = {
    "PMGControlsEditedDec2021": "control",  # Healthy cases
    "PMGstudycaseslabelled": "polymicrogyria"  # PMG cases
}

if os.path.exists(ppmr_path) and os.path.isdir(ppmr_path):
    print(f"Processing PPMR directory: {ppmr_path}")
    for main_dir, label_name in main_dir_labels.items():
        main_dir_path = os.path.join(ppmr_path, main_dir)

        if os.path.isdir(main_dir_path):
            print(f"Processing main directory: {main_dir_path}")
            for patient_dir in os.listdir(main_dir_path):
                patient_path = os.path.join(main_dir_path, patient_dir)

                if os.path.isdir(patient_path):
                    for sub_dir in os.listdir(patient_path):
                        sub_dir_path = os.path.join(patient_path, sub_dir)

                        if os.path.isdir(sub_dir_path):
                            for fname in os.listdir(sub_dir_path):
                                if fname.endswith(('.jpg', '.png', '.jpeg')):  # Image extensions
                                    file_path = os.path.join(sub_dir_path, fname)
                                    image_paths.append(file_path)
                                    labels.append(label_name)
        else:
            print(f"Main directory not found: {main_dir_path}")
else:
    print("PPMR directory not found.")

# Check if any files were found
print(f"Total images found: {len(image_paths)}")

# Label encoding
if image_paths:
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)

    # Split dataset into train and test
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        image_paths, encoded_labels, test_size=0.2, random_state=42
    )
    print(f"Train set: {len(train_paths)}, Test set: {len(test_paths)}")
else:
    print("No images found. Ensure the directory structure and file extensions are correct.")

# Data Generator for Loading and Preprocessing Images
def image_generator(image_paths, labels, batch_size=32, target_size=(224, 224)):
    while True:
        for start in range(0, len(image_paths), batch_size):
            end = min(start + batch_size, len(image_paths))
            batch_paths = image_paths[start:end]
            batch_labels = labels[start:end]

            # Load and preprocess images
            images = []
            for path in batch_paths:
                img = load_img(path, target_size=target_size)
                img_array = img_to_array(img) / 255.0  # Normalize pixel values
                images.append(img_array)

            yield np.array(images), np.array(batch_labels)

# Create train and validation generators
train_generator = image_generator(train_paths, train_labels, batch_size=32)
val_generator = image_generator(test_paths, test_labels, batch_size=32)

# Define Model (e.g., ResNet50, MobileNetV2, or EfficientNetB0)
def build_model(base_model):
    base_model.trainable = False  # Freeze the base model layers

    inputs = layers.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)

    # Add custom CNN layers
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(1, activation='sigmoid')(x)  # Binary classification: 1 output neuron with sigmoid

    model = models.Model(inputs, x)
    return model

# Choose the pre-trained model (ResNet50 as an example)
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Build the model
model = build_model(base_model)

# Compile the model with recall as a metric (using binary_crossentropy)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Recall()])  # For binary classification

# Define EarlyStopping callback (patience in terms of epochs)
batch_size = 32
max_iterations = 4500
iterations_per_epoch = len(train_paths) // batch_size
epochs_until_stop = max_iterations // iterations_per_epoch

early_stopping = EarlyStopping(monitor='val_loss', patience=epochs_until_stop, restore_best_weights=True)

# Train the model with EarlyStopping
model.fit(
    train_generator,
    validation_data=val_generator,
    steps_per_epoch=len(train_paths) // batch_size,  # Steps per epoch
    validation_steps=len(test_paths) // batch_size,  # Validation steps
    epochs=400,  # Maximum number of epochs
    callbacks=[early_stopping]
)

# Evaluate the model using Recall
val_loss, val_recall = model.evaluate(val_generator, steps=len(test_paths) // batch_size)
print(f"Validation Recall: {val_recall * 100:.2f}%")


Processing PPMR directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR
Processing main directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR/PMGControlsEditedDec2021
Processing main directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR/PMGstudycaseslabelled
Total images found: 15056
Train set: 12044, Test set: 3012
Epoch 1/400
376/376 ━━━━━━━━━━━━━━━━━━━━ 107s 224ms/step - loss: 0.5811 - recall: 0.2294 - val_loss: 0.1928 - val_recall: 0.8499
Epoch 2/400
376/376 ━━━━━━━━━━━━━━━━━━━━ 85s 215ms/step - loss: 0.1754 - recall: 0.8538 - val_loss: 0.1545 - val_recall: 0.8115
Epoch 3/400
376/376 ━━━━━━━━━━━━━━━━━━━━ 75s 199ms/step - loss: 0.1116 - recall: 0.9118 - val_loss: 0.0708 - val_recall: 0.9200
Epoch 4/400
376/376 ━━━━━━━━━━━━━━━━━━━━ 75s 198ms/step - loss: 0.0782 - recall: 0.9368 - val_loss: 0.0519 - val_recall: 0.9400
Epo

KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import Recall

# Define base dataset path
base_path = '/root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2'
ppmr_path = os.path.join(base_path, 'PPMR')

# Collect image paths and labels
image_paths = []
labels = []

# Define a mapping for directories
main_dir_labels = {
    "PMGControlsEditedDec2021": "control",  # Healthy cases
    "PMGstudycaseslabelled": "polymicrogyria"  # PMG cases
}

if os.path.exists(ppmr_path) and os.path.isdir(ppmr_path):
    print(f"Processing PPMR directory: {ppmr_path}")
    for main_dir, label_name in main_dir_labels.items():
        main_dir_path = os.path.join(ppmr_path, main_dir)

        if os.path.isdir(main_dir_path):
            print(f"Processing main directory: {main_dir_path}")
            for patient_dir in os.listdir(main_dir_path):
                patient_path = os.path.join(main_dir_path, patient_dir)

                if os.path.isdir(patient_path):
                    for sub_dir in os.listdir(patient_path):
                        sub_dir_path = os.path.join(patient_path, sub_dir)

                        if os.path.isdir(sub_dir_path):
                            for fname in os.listdir(sub_dir_path):
                                if fname.endswith(('.jpg', '.png', '.jpeg')):  # Image extensions
                                    file_path = os.path.join(sub_dir_path, fname)
                                    image_paths.append(file_path)
                                    labels.append(label_name)
        else:
            print(f"Main directory not found: {main_dir_path}")
else:
    print("PPMR directory not found.")

# Check if any files were found
print(f"Total images found: {len(image_paths)}")

# Label encoding
if image_paths:
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)

    # Split dataset into train and test
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        image_paths, encoded_labels, test_size=0.2, random_state=42
    )
    print(f"Train set: {len(train_paths)}, Test set: {len(test_paths)}")
else:
    print("No images found. Ensure the directory structure and file extensions are correct.")

# Data Generator for Loading and Preprocessing Images
def image_generator(image_paths, labels, batch_size=32, target_size=(224, 224)):
    while True:
        for start in range(0, len(image_paths), batch_size):
            end = min(start + batch_size, len(image_paths))
            batch_paths = image_paths[start:end]
            batch_labels = labels[start:end]

            # Load and preprocess images
            images = []
            for path in batch_paths:
                img = load_img(path, target_size=target_size)
                img_array = img_to_array(img) / 255.0  # Normalize pixel values
                images.append(img_array)

            yield np.array(images), np.array(batch_labels)

# Create train and validation generators
train_generator = image_generator(train_paths, train_labels, batch_size=32)
val_generator = image_generator(test_paths, test_labels, batch_size=32)

# Define Model (e.g., ResNet50, MobileNetV2, or EfficientNetB0)
def build_model(base_model):
    base_model.trainable = False  # Freeze the base model layers

    inputs = layers.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)

    # Add custom CNN layers
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(2, activation='softmax')(x)  # Binary classification (polymicrogyria vs control)

    model = models.Model(inputs, x)
    return model

# Choose the pre-trained model (ResNet50 as an example)
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Build the model
model = build_model(base_model)

# Compile the model with recall as a metric
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[Recall()])  # Use Recall instead of Accuracy

# Define EarlyStopping callback (patience in terms of epochs)
batch_size = 32
max_iterations = 4500
iterations_per_epoch = len(train_paths) // batch_size
epochs_until_stop = max_iterations // iterations_per_epoch

early_stopping = EarlyStopping(monitor='val_loss', patience=epochs_until_stop, restore_best_weights=True)

# Train the model with EarlyStopping
model.fit(
    train_generator,
    validation_data=val_generator,
    steps_per_epoch=len(train_paths) // batch_size,  # Steps per epoch
    validation_steps=len(test_paths) // batch_size,  # Validation steps
    epochs=400,  # Maximum number of epochs
    callbacks=[early_stopping]
)

# Evaluate the model using Recall
val_loss, val_recall = model.evaluate(val_generator, steps=len(test_paths) // batch_size)
print(f"Validation Recall: {val_recall * 100:.2f}%")



Processing PPMR directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR
Processing main directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR/PMGControlsEditedDec2021
Processing main directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR/PMGstudycaseslabelled
Total images found: 15056
Train set: 12044, Test set: 3012
Epoch 1/400


InvalidArgumentError: Graph execution error:

Detected at node LogicalAnd defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [1,32] vs. [1,64]
	 [[{{node LogicalAnd}}]]
	tf2xla conversion failed while converting __inference_one_step_on_data_64414[]. Run with TF_DUMP_GRAPH_PREFIX=/path/to/dump/dir and --vmodule=xla_compiler=2 to obtain a dump of the compiled functions.
	 [[StatefulPartitionedCall]] [Op:__inference_one_step_on_iterator_65103]

In [ ]:
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch

# Step 1: Clone NKUT repository and install dependencies
os.system("git clone https://github.com/nkicsl/NKUT")
os.chdir("NKUT")
os.system("pip install -r requirements.txt")

# Step 2: Define dataset paths and preprocessing
base_path = "/path/to/PPMR"  # Update this with the path to your PPMR dataset
image_paths = []
labels = []

# Mapping for directories in PPMR
main_dir_labels = {
    "PMGControlsEditedDec2021": "control",
    "PMGstudycaseslabelled": "polymicrogyria"
}

# Collect image paths and labels
for main_dir, label_name in main_dir_labels.items():
    main_dir_path = os.path.join(base_path, main_dir)

    if os.path.isdir(main_dir_path):
        for patient_dir in os.listdir(main_dir_path):
            patient_path = os.path.join(main_dir_path, patient_dir)

            if os.path.isdir(patient_path):
                for sub_dir in os.listdir(patient_path):
                    sub_dir_path = os.path.join(patient_path, sub_dir)

                    if os.path.isdir(sub_dir_path):
                        for fname in os.listdir(sub_dir_path):
                            if fname.endswith(('.jpg', '.jpeg', '.png')):
                                image_paths.append(os.path.join(sub_dir_path, fname))
                                labels.append(label_name)

# Check for images
if not image_paths:
    raise ValueError("No images found. Check dataset structure or file extensions.")

# Step 3: Encode labels and split the dataset
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, encoded_labels, test_size=0.2, random_state=42
)

# Step 4: Define dataset class
class PPMRDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

# Step 5: Define transformations
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Step 6: Create DataLoaders
train_dataset = PPMRDataset(train_paths, train_labels, transform=train_transform)
test_dataset = PPMRDataset(test_paths, test_labels, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 7: Integrate DataLoaders into NKUT framework
# Import and integrate with NKUT model training script
import nkut  # Replace with actual NKUT training script/module if applicable

model = nkut.NKUTModel()  # Placeholder for NKUT model initialization
model.train(train_loader, test_loader)  # Adjust with actual NKUT training functions

# Step 8: Verify the pipeline
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")


ValueError: No images found. Check dataset structure or file extensions.

In [ ]:
import os

# Base directory of the dataset
base_path = "/path/to/PPMR"  # Replace this with the actual path to your PPMR dataset

image_paths = []
labels = []

# Mapping main directories to labels
main_dir_labels = {
    "PMGControlsEditedDec2021": "control",
    "PMGstudycaseslabelled": "polymicrogyria"
}

# Collecting image paths and labels
for main_dir, label_name in main_dir_labels.items():
    main_dir_path = os.path.join(base_path, main_dir)

    if os.path.isdir(main_dir_path):
        for patient_dir in os.listdir(main_dir_path):
            patient_path = os.path.join(main_dir_path, patient_dir)

            if os.path.isdir(patient_path):
                for sub_dir in os.listdir(patient_path):
                    sub_dir_path = os.path.join(patient_path, sub_dir)

                    if os.path.isdir(sub_dir_path):
                        for file_name in os.listdir(sub_dir_path):
                            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                                image_paths.append(os.path.join(sub_dir_path, file_name))
                                labels.append(label_name)

# Check for images
if not image_paths:
    print(f"No images found in the directory: {base_path}")
else:
    print(f"Found {len(image_paths)} images.")
    print(f"Sample image: {image_paths[0]}")


No images found in the directory: /path/to/PPMR


In [ ]:
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch

# Step 1: Define dataset base path
base_path = "/path/to/PPMR"  # Replace with the actual root path of your dataset

# Step 2: Initialize lists for image paths and labels
image_paths = []
labels = []

# Mapping for directories in PPMR
main_dir_labels = {
    "PMGControlsEditedDec2021": "control",
    "PMGstudycaseslabelled": "polymicrogyria"
}

# Step 3: Collect image paths and labels
for main_dir, label_name in main_dir_labels.items():
    main_dir_path = os.path.join(base_path, main_dir)

    if os.path.isdir(main_dir_path):
        for patient_dir in os.listdir(main_dir_path):
            patient_path = os.path.join(main_dir_path, patient_dir)

            if os.path.isdir(patient_path):
                for fname in os.listdir(patient_path):
                    if fname.endswith(('.jpg', '.jpeg', '.png')):
                        file_path = os.path.join(patient_path, fname)
                        image_paths.append(file_path)
                        labels.append(label_name)
    else:
        print(f"Directory not found: {main_dir_path}")

# Step 4: Check for images
if not image_paths:
    raise ValueError("No images found. Check dataset structure or file extensions.")

# Step 5: Encode labels and split the dataset
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, encoded_labels, test_size=0.2, random_state=42
)

# Step 6: Define dataset class
class PPMRDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

# Step 7: Define transformations
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Step 8: Create DataLoaders
train_dataset = PPMRDataset(train_paths, train_labels, transform=train_transform)
test_dataset = PPMRDataset(test_paths, test_labels, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 9: Verify
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")


Directory not found: /path/to/PPMR/PMGControlsEditedDec2021
Directory not found: /path/to/PPMR/PMGstudycaseslabelled


ValueError: No images found. Check dataset structure or file extensions.

In [ ]:
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch

# Define the base path to the dataset
base_path = "/root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2"  # Update this to the actual path where your dataset resides

# Define a mapping for the main directories
main_dir_labels = {
    "PMGControlsEditedDec2021": "control",  # Healthy cases
    "PMGstudycaseslabelled": "polymicrogyria"  # PMG cases
}

# Collect image paths and labels
image_paths = []
labels = []

# Traverse directories and gather image data
if os.path.exists(base_path) and os.path.isdir(base_path):
    print(f"Processing base directory: {base_path}")
    for main_dir, label_name in main_dir_labels.items():
        main_dir_path = os.path.join(base_path, main_dir)

        if os.path.isdir(main_dir_path):
            print(f"Processing main directory: {main_dir_path}")
            for patient_dir in os.listdir(main_dir_path):
                patient_path = os.path.join(main_dir_path, patient_dir)

                if os.path.isdir(patient_path):
                    for sub_dir in os.listdir(patient_path):
                        sub_dir_path = os.path.join(patient_path, sub_dir)

                        if os.path.isdir(sub_dir_path):
                            for fname in os.listdir(sub_dir_path):
                                if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                                    file_path = os.path.join(sub_dir_path, fname)
                                    image_paths.append(file_path)
                                    labels.append(label_name)
else:
    print(f"Base directory not found or invalid: {base_path}")

# Check if any files were found
if not image_paths:
    raise ValueError("No images found. Ensure the directory structure and file extensions are correct.")
else:
    print(f"Total images found: {len(image_paths)}")
    print(f"Sample image: {image_paths[0]}")

# Label encoding and dataset splitting
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, encoded_labels, test_size=0.2, random_state=42
)
print(f"Train set: {len(train_paths)}, Test set: {len(test_paths)}")

# Define dataset class
class PPMRDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create DataLoaders
train_dataset = PPMRDataset(train_paths, train_labels, transform=train_transform)
test_dataset = PPMRDataset(test_paths, test_labels, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print verification information
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

# Example of using a batch from train_loader
data_iter = iter(train_loader)
images, labels = next(data_iter)

print(f"Batch size: {images.size(0)}")


Processing base directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2


ValueError: No images found. Ensure the directory structure and file extensions are correct.

In [ ]:
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch

# Updated base path from your code
base_path = '/root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2'
ppmr_path = os.path.join(base_path, 'PPMR')

# Define a mapping for the main directories
main_dir_labels = {
    "PMGControlsEditedDec2021": "control",  # Healthy cases
    "PMGstudycaseslabelled": "polymicrogyria"  # PMG cases
}

# Collect image paths and labels
image_paths = []
labels = []

# Traverse directories and gather image data
if os.path.exists(ppmr_path) and os.path.isdir(ppmr_path):
    print(f"Processing PPMR directory: {ppmr_path}")
    for main_dir, label_name in main_dir_labels.items():
        main_dir_path = os.path.join(ppmr_path, main_dir)

        if os.path.isdir(main_dir_path):
            print(f"Processing main directory: {main_dir_path}")
            for patient_dir in os.listdir(main_dir_path):
                patient_path = os.path.join(main_dir_path, patient_dir)

                if os.path.isdir(patient_path):
                    for sub_dir in os.listdir(patient_path):
                        sub_dir_path = os.path.join(patient_path, sub_dir)

                        if os.path.isdir(sub_dir_path):
                            for fname in os.listdir(sub_dir_path):
                                if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                                    file_path = os.path.join(sub_dir_path, fname)
                                    image_paths.append(file_path)
                                    labels.append(label_name)
else:
    print(f"Base directory not found or invalid: {ppmr_path}")

# Check if any files were found
if not image_paths:
    raise ValueError("No images found. Ensure the directory structure and file extensions are correct.")
else:
    print(f"Total images found: {len(image_paths)}")
    print(f"Sample image: {image_paths[0]}")

# Label encoding and dataset splitting
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, encoded_labels, test_size=0.2, random_state=42
)
print(f"Train set: {len(train_paths)}, Test set: {len(test_paths)}")

# Define dataset class
class PPMRDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create DataLoaders
train_dataset = PPMRDataset(train_paths, train_labels, transform=train_transform)
test_dataset = PPMRDataset(test_paths, test_labels, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print verification information
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

# Example of using a batch from train_loader
data_iter = iter(train_loader)
images, labels = next(data_iter)

print(f"Batch size: {images.size(0)}")


Processing PPMR directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR
Processing main directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR/PMGControlsEditedDec2021
Processing main directory: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR/PMGstudycaseslabelled
Total images found: 15056
Sample image: /root/.cache/kagglehub/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset/versions/2/PPMR/PMGControlsEditedDec2021/4/4control2/4control2_cor_0_111.jpg
Train set: 12044, Test set: 3012
Number of training samples: 12044
Number of testing samples: 3012
Batch size: 32


In [ ]:
from sklearn.metrics import recall_score

def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        start_time = time.time()
        running_loss = 0.0
        correct = 0
        total = 0

        print(f"Epoch {epoch + 1}/{num_epochs}")
        with tqdm(total=len(train_loader), desc="Training") as pbar:
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                correct += torch.sum(preds == labels.data)
                total += labels.size(0)

                pbar.update(1)

        scheduler.step()

        epoch_loss = running_loss / total
        epoch_acc = correct.double() / total
        epoch_time = time.time() - start_time

        print(f"Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, Time: {epoch_time:.2f}s")

def evaluate_model(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    print("Evaluating...")
    with tqdm(total=len(test_loader), desc="Evaluation") as pbar:
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)

                # Collect predictions and true labels for recall calculation
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                pbar.update(1)

    loss = running_loss / len(test_loader.dataset)

    # Calculate recall using sklearn
    recall = recall_score(all_labels, all_preds, average="binary")  # Binary classification
    print(f"Test Loss: {loss:.4f}, Test Recall: {recall:.4f}")
    return loss, recall

# Step 4: Train and evaluate the model
print("Starting training...")
train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=1)

print("Evaluating on the test set...")
evaluate_model(model, test_loader, criterion)


Starting training...


NameError: name 'criterion' is not defined

In [ ]:
import torch
from torch import nn, optim
from torchvision import models
from sklearn.metrics import recall_score
from tqdm import tqdm
import time

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to train the model
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        start_time = time.time()
        running_loss = 0.0
        correct = 0
        total = 0

        print(f"Epoch {epoch + 1}/{num_epochs}")
        with tqdm(total=len(train_loader), desc="Training") as pbar:
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                correct += torch.sum(preds == labels.data)
                total += labels.size(0)

                pbar.update(1)

        scheduler.step()

        epoch_loss = running_loss / total
        epoch_acc = correct.double() / total
        epoch_time = time.time() - start_time

        print(f"Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, Time: {epoch_time:.2f}s")

# Function to evaluate the model
def evaluate_model(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    print("Evaluating...")
    with tqdm(total=len(test_loader), desc="Evaluation") as pbar:
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)

                # Collect predictions and true labels for recall calculation
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                pbar.update(1)

    loss = running_loss / len(test_loader.dataset)

    # Calculate recall using sklearn
    recall = recall_score(all_labels, all_preds, average="binary")  # Binary classification
    print(f"Test Loss: {loss:.4f}, Test Recall: {recall:.4f}")
    return recall

# Initialize models
models_to_train = {
    "ResNet50": models.resnet50(pretrained=True),
    "MobileNetV2": models.mobilenet_v2(pretrained=True)
}

# Adjust final layers for binary classification
for model_name, model in models_to_train.items():
    if hasattr(model, "fc"):  # For ResNet
        model.fc = nn.Linear(model.fc.in_features, 2)
    elif hasattr(model, "classifier"):  # For MobileNetV2
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    models_to_train[model_name] = model.to(device)

# Define training parameters
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
num_epochs = 1

# Train and evaluate each model
for model_name, model in models_to_train.items():
    print(f"\nTraining and Evaluating: {model_name}")
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

    train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs)
    recall = evaluate_model(model, test_loader, criterion)
    print(f"Model: {model_name}, Recall: {recall:.4f}")



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 170MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed i


Training and Evaluating: ResNet50
Epoch 1/1


Training: 100%|██████████| 377/377 [14:31<00:00,  2.31s/it]


Loss: 0.1119, Accuracy: 0.9702, Time: 871.19s
Evaluating...


Evaluation: 100%|██████████| 95/95 [00:38<00:00,  2.48it/s]


Test Loss: 1.5132, Test Recall: 0.5960
Model: ResNet50, Recall: 0.5960

Training and Evaluating: MobileNetV2
Epoch 1/1


Training: 100%|██████████| 377/377 [12:38<00:00,  2.01s/it]


Loss: 0.0562, Accuracy: 0.9818, Time: 758.81s
Evaluating...


Evaluation: 100%|██████████| 95/95 [00:33<00:00,  2.85it/s]

Test Loss: 0.0284, Test Recall: 0.9955
Model: MobileNetV2, Recall: 0.9955


In [ ]:
import torch
from torch import nn, optim
from torchvision import models
from sklearn.metrics import recall_score
from tqdm import tqdm

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize EfficientNet-B0
efficientnet_b0 = models.efficientnet_b0(pretrained=True)

# Adjust the final layer for binary classification
efficientnet_b0.fc = nn.Linear(efficientnet_b0.fc.in_features, 2)
efficientnet_b0 = efficientnet_b0.to(device)

# Define loss function, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficientnet_b0.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

# Train the model for 1 epoch
import torch
from torch import nn, optim
from torchvision import models
from torchvision.models import EfficientNet_B0_Weights
from sklearn.metrics import recall_score
from tqdm import tqdm

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize EfficientNet-B0 with weights
efficientnet_b0 = models.efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

# Adjust the classifier for binary classification
efficientnet_b0.classifier[1] = nn.Linear(efficientnet_b0.classifier[1].in_features, 2)
efficientnet_b0 = efficientnet_b0.to(device)

# Define loss function, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficientnet_b0.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

# Train the model for 1 epoch
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        with tqdm(total=len(train_loader), desc="Training") as pbar:
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                pbar.update(1)
        scheduler.step()

# Evaluate the model and calculate recall
def evaluate_model(model, test_loader, criterion):
    model.eval()
    all_preds = []
    all_labels = []
    with tqdm(total=len(test_loader), desc="Evaluation") as pbar:
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                pbar.update(1)
    recall = recall_score(all_labels, all_preds, average="binary")
    print(f"Recall: {recall:.4f}")
    return recall

# Train and evaluate EfficientNet-B0
train_model(efficientnet_b0, train_loader, criterion, optimizer, scheduler, num_epochs=1)
evaluate_model(efficientnet_b0, test_loader, criterion)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 75.0MB/s]


AttributeError: 'EfficientNet' object has no attribute 'fc'

In [ ]:
import torch
from torch import nn, optim
from torchvision import models
from torchvision.models import EfficientNet_B0_Weights
from sklearn.metrics import recall_score
from tqdm import tqdm

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize EfficientNet-B0 with weights
efficientnet_b0 = models.efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

# Adjust the classifier for binary classification
efficientnet_b0.classifier[1] = nn.Linear(efficientnet_b0.classifier[1].in_features, 2)
efficientnet_b0 = efficientnet_b0.to(device)

# Define loss function, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficientnet_b0.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

# Train the model for 1 epoch
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        with tqdm(total=len(train_loader), desc="Training") as pbar:
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                pbar.update(1)
        scheduler.step()

# Evaluate the model and calculate recall
def evaluate_model(model, test_loader, criterion):
    model.eval()
    all_preds = []
    all_labels = []
    with tqdm(total=len(test_loader), desc="Evaluation") as pbar:
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                pbar.update(1)
    recall = recall_score(all_labels, all_preds, average="binary")
    print(f"Recall: {recall:.4f}")
    return recall

# Train and evaluate EfficientNet-B0
train_model(efficientnet_b0, train_loader, criterion, optimizer, scheduler, num_epochs=1)
evaluate_model(efficientnet_b0, test_loader, criterion)


Epoch 1/1


Evaluation: 100%|██████████| 95/95 [00:34<00:00,  2.79it/s]

Recall: 1.0000


1.0

In [ ]:
import torch
from torch import nn
from torchvision import models
from torchvision.models import ResNet50_Weights, MobileNet_V2_Weights, EfficientNet_B0_Weights
from sklearn.metrics import recall_score
from tqdm import tqdm

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize individual models with pretrained weights
resnet50 = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
mobilenet_v2 = models.mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
efficientnet_b0 = models.efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

# Adjust final layers for binary classification
resnet50.fc = nn.Linear(resnet50.fc.in_features, 2)
mobilenet_v2.classifier[1] = nn.Linear(mobilenet_v2.classifier[1].in_features, 2)
efficientnet_b0.classifier[1] = nn.Linear(efficientnet_b0.classifier[1].in_features, 2)

# Move models to device
resnet50 = resnet50.to(device)
mobilenet_v2 = mobilenet_v2.to(device)
efficientnet_b0 = efficientnet_b0.to(device)

# Define a function to train individual models
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        with tqdm(total=len(train_loader), desc="Training") as pbar:
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                pbar.update(1)
        scheduler.step()

# Train each model for 1 epoch
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001

for model, model_name in zip(
    [resnet50, mobilenet_v2, efficientnet_b0], ["ResNet50", "MobileNetV2", "EfficientNet-B0"]
):
    print(f"\nTraining {model_name}...")
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
    train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=1)

# Define the ensemble function
def ensemble_evaluate(models, test_loader):
    all_preds = []
    all_labels = []

    print("\nEvaluating Ensemble...")
    with tqdm(total=len(test_loader), desc="Ensemble Evaluation") as pbar:
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                # Average predictions from all models
                outputs = torch.stack([model(inputs) for model in models], dim=0)
                avg_outputs = torch.mean(outputs, dim=0)

                _, preds = torch.max(avg_outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                pbar.update(1)

    # Calculate recall score
    recall = recall_score(all_labels, all_preds, average="binary")
    print(f"Ensemble Recall: {recall:.4f}")
    return recall

# Evaluate the ensemble model
ensemble_recall = ensemble_evaluate([resnet50, mobilenet_v2, efficientnet_b0], test_loader)
